In [1]:
import numpy as np 
import pandas as pd
import math as mt

In [2]:
df = pd.read_csv("C:\\Users\\abhic\\Data_Mining\\weather1.csv")

In [3]:
df.drop('day', axis = 1,inplace=True)
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [4]:
X = df.drop('play', axis = 1)
y = df['play']

In [5]:
def entropy(X, cname='play'):
    entropy = 0
    value = X[cname].unique()
    for v in value:
        fraction = X[cname].value_counts()[v]/len(X)
        temp = -fraction * mt.log(fraction, 2)
        entropy += temp
    return entropy

In [6]:
entropy(df)

0.9402859586706309

In [7]:
col = df.columns
col = list(col)
col.remove('play')

In [8]:
def entropyAll(df, col, target='play'):
    #col = df.columns
    E = entropy(df, target)
    mval = 0
    mname = target
    for cname in col:
        x = 0
        value = df[cname].unique()
        for v in value:
            t = df[df[cname] == v]
            x += entropy(t) * len(t)/len(df)
        Gain = E - x
        #print("Entropy of ",cname," = ",x)
        if(Gain > mval):
            mval = x
            mname = cname
    return mname

In [9]:
def subData(df, mname, value):
    return df[df[mname] == value]

In [10]:
def decisionTree(df, target = 'play', tree=None):
    col = list(df.columns)
    col.remove(target)
    root = entropyAll(df, col, target)
    val = np.unique(df[root])
    if tree is None:
        tree={}
        tree[root] = {}
    for i in val:
        SubData = subData(df,root,i)
        v,counts = np.unique(SubData[target],return_counts=True) 
        if len(counts)==1:
            tree[root][i] = v[0]                                                    
        else:
            tree[root][i] = decisionTree(SubData,target)
    return tree

In [11]:
T = decisionTree(df)
print(T)

{'outlook': {'Overcast': 'Yes', 'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'wind': {'Strong': {'humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Weak': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}}}


In [12]:
def onePredict(x,t):
    for k in t:
        k = str(k)
        v = x[k]
        t = t[k][v]
        if type(t)!= dict:
            return(t)
        out = onePredict(x,t)
        return out

In [13]:
def allPredict(X,T):
    n = len(X)
    output = []
    for i in range(n):
        t = T
        x = X.iloc[i]
        out = onePredict(x,t)
        #print(out)
        output.append(out)
    return output

In [14]:
allPredict(X, T)

['No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No']